In [7]:
import cv2
import numpy as np
import glob

In [8]:
# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*7,3), np.float32)

# objp 产生一个三维世界的点，因为是棋盘所以z轴为0
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
# print(objp)

In [19]:
x = 0
images = glob.glob('/Users/triste/MarchingOn/project/stereo/left/*.jpg')
# print(images)
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (7,6), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        # 精细化角点信息
        corners2 = cv2.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners)
        # Draw and display the corners
        # 绘制查找到的角点
#         cv2.drawChessboardCorners(img, (7,6), corners2, ret)
#         cv2.imshow('img', img)
#         cv2.waitKey(500)
# cv2.destroyAllWindows()
# 标定，mtx 是相机内参，dist 是畸变，rvecs,tvecs 分别是旋转矩阵和平移矩阵代表外参
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

In [21]:
img = cv2.imread('/Users/triste/MarchingOn/project/stereo/left/left12.jpg')
h,  w = img.shape[:2]
# 基于自由缩放参数来细化相机矩阵。返回图像ROI，可用于裁剪去畸变图片。
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))
dst = cv2.undistort(img, mtx, dist, None, newcameramtx)
# 裁剪图像
x, y, w, h = roi
# dst = dst[y:y + h, x:x + w]
cv2.imwrite('/Users/triste/MarchingOn/project/stereo/left/result_no_crop.jpg', dst)

True

In [23]:
mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv2.norm(imgpoints[i], imgpoints2, cv2.NORM_L2)/len(imgpoints2)
    mean_error += error
print( "total error: {}".format(mean_error/len(objpoints)) )

total error: 0.023686000375385673
